In [72]:
import numpy as np

dir_to_number = {
    "^": 2,
    "v": 3,
    "<": 4,
    ">": 5
}
number_to_dir = {v: k for k, v in dir_to_number.items()}

# rotate +90 degrees
dir_to_new_dir = {"^": ">", ">": "v", "v": "<", "<": "^"}

def parse_input(fname):
    with open(fname, 'r') as file:
        text = file.read()
        text = text.replace(".", "0")
        text = text.replace("#", "1")
        for dir,number in dir_to_number.items():
            text = text.replace(dir, str(number))

        lines = text.splitlines()
    n_rows = len(lines)
    n_cols = len(lines[0])
    print(n_rows, n_cols)
    # transform to numpy array of integers
    grid = np.zeros((n_rows, n_cols), dtype=int)
    for i in range(n_rows):
        for j in range(n_cols):
            grid[i, j] = int(lines[i][j])
    return grid



def find_start(grid):
    mask = grid > 1
    start_coords = np.where(mask)
    start_dir_number = grid[start_coords]
    start_dir = number_to_dir[start_dir_number[0]]

    start_coords = (start_coords[0][0], start_coords[1][0])
    return start_coords, start_dir

def move(grid, start_coords, start_dir):
    direction = start_dir
    # assert coords are within the grid
    assert 0 <= start_coords[0] < grid.shape[0]
    assert 0 <= start_coords[1] < grid.shape[1]
    if direction == "^":
        trajectory = grid[:start_coords[0], start_coords[1]][::-1]
    elif direction == "v":
        trajectory = grid[start_coords[0]+1:, start_coords[1]]
    elif direction == "<":
        trajectory = grid[start_coords[0], :start_coords[1]][::-1]
    elif direction == ">":
        trajectory = grid[start_coords[0], start_coords[1]+1:]
    # steps up to the first obstacle (without the obstacle)
    # if there is no obstacle, n_steps is None
    n_steps = np.nonzero(trajectory == 1)[0]
    if len(n_steps) == 0:
        # no obstacles
        n_steps = len(trajectory)
        new_dir = None
    else:
        # first obstacle is at n_steps[0]
        n_steps = n_steps[0]
        new_dir = dir_to_new_dir[direction]
    if direction == "^":
        new_coords = (start_coords[0] - n_steps, start_coords[1])
    elif direction == "v":
        new_coords = (start_coords[0] + n_steps, start_coords[1])
    elif direction == "<":
        new_coords = (start_coords[0], start_coords[1] - n_steps)
    elif direction == ">":
        new_coords = (start_coords[0], start_coords[1] + n_steps)
    return new_coords, new_dir, n_steps



def get_visited_coords(grid, start_coords, start_dir, verbose=False):
    coords, direction = start_coords, start_dir
    total_steps = 0
    visited_coords = set()
    taken_instructions = set()
    while direction is not None:
        old_coords = coords
        old_dir = direction
        coords, direction, n_steps = move(grid, coords, direction)
        instructions = (old_coords, old_dir, n_steps)
        # print instructions with nice formatting
        if verbose:
            print(f"{old_coords} {old_dir} {n_steps}")
        if instructions in taken_instructions:
            if verbose:
                print("loop detected")
            return None, visited_coords
        else:
            taken_instructions.add(instructions)
        total_steps += n_steps
        for i in range(n_steps+1):
            if old_dir == "^":
                coords = (old_coords[0] - i, old_coords[1])
            elif old_dir == "v":
                coords = (old_coords[0] + i, old_coords[1])
            elif old_dir == "<":
                coords = (old_coords[0], old_coords[1] - i)
            elif old_dir == ">":
                coords = (old_coords[0], old_coords[1] + i)
            visited_coords.add(coords)
    return total_steps, visited_coords



In [73]:
grid = parse_input("input_small.txt")
print(grid)

start_coords, start_dir = find_start(grid)
total_steps, visited_coords = get_visited_coords(grid, start_coords, start_dir)
print(total_steps)
print(len(visited_coords))

10 10
[[0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]]
44
41


## Second Part

In [ ]:
def check_loop(grid, start_coords, start_dir, obstacle_coords):
    grid = grid.copy()
    grid[obstacle_coords] = 1

    total_steps, visited_coords = get_visited_coords(grid, start_coords, start_dir)
    return total_steps is None


grid = parse_input("input.txt")


start_coords, start_dir = find_start(grid)

# find all visited coordinates
total_steps, visited_coords = get_visited_coords(grid, start_coords, start_dir)
possible_obstacle_coords = visited_coords - set([start_coords])

# brute force check if there is a loop for all possible obstacle coordinates
n_loops = 0
loop_obstacle_coords = set()
for obstacle_coords in possible_obstacle_coords:
    is_loop = check_loop(grid, start_coords, start_dir, obstacle_coords)
    if is_loop:

        loop_obstacle_coords.add(obstacle_coords)
        n_loops += 1



print(n_loops)
print(loop_obstacle_coords)

130 130
1796
{(107, 95), (59, 55), (43, 110), (76, 107), (110, 20), (100, 92), (58, 95), (69, 104), (40, 77), (41, 42), (81, 78), (39, 117), (12, 100), (72, 114), (48, 22), (107, 108), (57, 107), (17, 71), (49, 103), (69, 81), (60, 32), (5, 96), (92, 101), (58, 108), (62, 78), (73, 87), (20, 81), (31, 90), (78, 22), (59, 45), (45, 110), (98, 36), (63, 15), (76, 97), (28, 57), (101, 41), (111, 91), (115, 61), (42, 77), (108, 22), (105, 53), (75, 110), (59, 22), (72, 104), (90, 9), (107, 98), (100, 59), (5, 50), (92, 55), (42, 90), (74, 42), (20, 71), (85, 51), (86, 16), (97, 25), (43, 54), (53, 104), (59, 35), (65, 78), (57, 74), (76, 87), (68, 83), (69, 48), (80, 57), (26, 86), (100, 72), (49, 106), (90, 107), (69, 84), (91, 108), (84, 69), (35, 27), (114, 91), (67, 16), (78, 25), (119, 26), (46, 78), (5, 40), (92, 45), (57, 87), (76, 100), (49, 83), (15, 90), (80, 70), (31, 34), (42, 80), (20, 61), (72, 71), (96, 56), (25, 112), (99, 61), (100, 26), (45, 90), (56, 99), (69, 38), (80, 